In [1]:
import json
import pandas as pd
import string
import regex as re
import pickle
import math

from pathlib import Path
from thoth.lab import common
from thoth.lab import solver
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
#from pyonmttok import Tokenizer
from nltk.corpus import stopwords
from string import punctuation

In [2]:
pd.set_option('max_colwidth', 800)

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/skotak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
FILE_NAME = "thoth-solver-dataset-v1.0.zip"
#common.extract_zip_file(FILE_NAME)

In [5]:
solver_reports = solver.aggregate_solver_results()
#solver_reports = solver.aggregate_solver_results(is_local=False)

INFO:thoth.lab.common:Number of file retrieved is: 10000


In [6]:
solver_reports_extracted_data = []
solver_errors = []
for solver_document in solver_reports:
    solver_report_extracted_data = solver.extract_data_from_solver_metadata(solver_document["metadata"])
    for k, v in solver_document["result"].items():
        solver_report_extracted_data[k] = v
        if k == "errors" and v:
            errors = solver.extract_errors_from_solver_result(v)
            for error in errors:
                solver_errors.append(error)
    
    packages = solver.extract_tree_from_solver_result(solver_document["result"])
    solver_report_extracted_data["packages"] = packages
    solver_reports_extracted_data.append(solver_report_extracted_data)


In [7]:
solver_total_errors_df = pd.DataFrame(solver_errors).reset_index()

solver_total_errors_df.head(4)

,index,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout
0,0,tryton,3.2.20,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps tryton==3.2.20 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe n...",1.0,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe not found: installers can not be created, ""\\n ...",Collecting tryton==3.2.20\n Downloading https://files.pythonhosted.org/packages/8d/b6/a88e6e4df1557cf3f265b12fd585f36e212fd8669aa394c9fbb5a86e56d3/tryton-3.2.20.tar.gz (811kB)\n,60.0
1,1,pyobjc-framework-contacts,4.0b1,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps pyobjc-framework-contacts==4.0b1 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/setup.py'""'""'; __file__='""'""'/tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/pip-egg-info\n cwd: /tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/\n Complete output (15 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/p...",1.0,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/setup.py'""'""'; __file__='""'""'/tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/pip-egg-info\n cwd: /tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/\n Complete output (15 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/...",Collecting pyobjc-framework-contacts==4.0b1

In [8]:
stop = punctuation + "``" + "''" + '""' + "/"
table = str.maketrans(stop, ' '*len(stop))

def split_log(log_messages):
    ids_with_different_log_pattern = []
    Error_info, command_info, cwd, Complete_output, ERROR, specific_error = {}, {}, {}, {}, {}, {}
    for idx, msg in enumerate(log_messages):
        msg = msg.translate(table)
        sentences = [x.strip() for x in msg.split('\n')]
        for id, sent in enumerate(sentences):
            if re.match(r"^Command.*", sent):
                Error_info[idx] = sent
            elif re.match(r"^command.*", sent):
                if idx in command_info.keys() and sent not in command_info[idx]:
                    command_info[idx].append(sent)
                else:
                    command_info[idx] = [sent]
            elif re.match(r"^cwd.*", sent):
                if idx in cwd.keys() and sent not in cwd[idx]:
                    cwd[idx].append(sent)
                else:
                    cwd[idx] = [sent]
            elif re.match(r"^Complete output.*", sent):
                number_of_lines = re.findall(r'\d+', sent) 
                if idx in Complete_output.keys() and sent not in Complete_output[idx]:
                    Complete_output[idx].append(sentences[id:id+int(number_of_lines[0])+1])
                else:
                    Complete_output[idx] = sentences[id:id+int(number_of_lines[0])+1]
            elif re.match(r".*Error.*", sent):
                sent = sent.split('.')
                if idx in specific_error.keys() and sent not in specific_error[idx]:
                    specific_error[idx].extend(sent)
                else:
                    specific_error[idx] = sent
            elif re.match(r"^ERROR.*", sent):
                if idx in ERROR.keys() and sent not in ERROR[idx]:
                    pass
                    ERROR[idx].append(sent)
                else:
                    ERROR[idx] = [sent]
                    
    return Error_info, command_info, cwd, Complete_output, ERROR, specific_error

print(len(solver_total_errors_df['message']))

1624


In [9]:
Error_info, command_info, cwd, Complete_output, ERROR, specific_error= split_log(solver_total_errors_df['message'])
print(len(Error_info), len(command_info), len(cwd), len(Complete_output), len(ERROR), len(specific_error))

1604 1233 1233 1233 1598 901


In [10]:
solver_total_errors_df['Error_info']= solver_total_errors_df['index'].map(Error_info)
solver_total_errors_df['command_info']= solver_total_errors_df['index'].map(command_info)
solver_total_errors_df['cwd']= solver_total_errors_df['index'].map(cwd)
solver_total_errors_df['Complete_output']= solver_total_errors_df['index'].map(Complete_output)
solver_total_errors_df['ERROR']= solver_total_errors_df['index'].map(ERROR)
solver_total_errors_df['specific_error']= solver_total_errors_df['index'].map(specific_error)

In [12]:
clustering_data = {}
for idx, error in enumerate(solver_total_errors_df['specific_error']):
    if type(error) != list:
        if pd.isnull(error) and  type(solver_total_errors_df.iloc[idx]['ERROR']) == list:
            #print(idx, solver_total_errors_df.iloc[idx]['ERROR'])
            clustering_data[idx] = [solver_total_errors_df.iloc[idx]['ERROR'][0]]
        if type(solver_total_errors_df.iloc[idx]['ERROR']) != list:
            if pd.isnull(solver_total_errors_df.iloc[idx]['ERROR']):
                clustering_data[idx] = [solver_total_errors_df.iloc[idx]['Error_info']]
                if type(solver_total_errors_df.iloc[idx]['Error_info']) != list:
                    if pd.isnull(solver_total_errors_df.iloc[idx]['Error_info']):
                        clustering_data[idx] = [solver_total_errors_df.iloc[idx]['message']]
    else:
        for sent in error:
            regex = re.compile(".*Error") 
            match_object = regex.findall(sent)
            if re.match(r"^Error.*", sent):
                clustering_data[idx] = [sent]
            elif match_object:
                clustering_data[idx] = match_object

In [13]:
clustering_data

{0: ['SyntaxError'],
 1: ['FileNotFoundError'],
 2: ['RuntimeError'],
 3: ['SyntaxError'],
 4: ['cffi VerificationError  CompileError'],
 5: ['ERROR  Failed building wheel for spacy'],
 6: ['ERROR  Failed building wheel for fastavro'],
 7: ['ERROR  Failed building wheel for pyeclib'],
 8: ['NameError'],
 9: ['SyntaxError'],
 10: ['SyntaxError'],
 11: ['ERROR  Failed building wheel for thriftrw'],
 12: ['ERROR  No matching distribution found for robotexclusionrulesparser  1 1'],
 13: ['FileNotFoundError'],
 14: ['SyntaxError'],
 15: ['FileNotFoundError'],
 16: ['ERROR  No matching distribution found for itk segmentation  4 13 0'],
 17: ['SyntaxError'],
 18: ['ERROR  No matching distribution found for mxnet mkl  1 6 0b20200217'],
 19: ['ModuleNotFoundError'],
 20: ['FileNotFoundError'],
 21: ['ERROR  No matching distribution found for wrapt  1 10 3'],
 22: ['FileNotFoundError'],
 23: ['FileNotFoundError'],
 24: ['FileNotFoundError'],
 25: ['ERROR  No matching distribution found for mutag

In [14]:
_line_number = r'(at line[:]*\s*\d+)'
_url = r'(http[s]|root|srm|file)*:(//|/)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
_filepath = "(/[a-zA-Z\./]*[\s]?)"
path_regex = re.compile(r'(\b\w+://)\S+(?=\s)')
file_regex = re.compile(r'(\b[f|F]ile( exists)?:?\s?)/\S+(?=\s)')
py_regex = re.compile(r'/?\b[-./_a-zA-Z0-9]+\.py\b')
long_regex = re.compile(r'[-/_a-zA-Z0-9]{25,}')

In [15]:
def remove_whitespaces(sentence):
    return " ".join(sentence.split())

def substitute_path(string):
    string = path_regex.sub(r'\1', string)
    string = py_regex.sub(r' ', string)
    string = file_regex.sub(r'\1', string)
    string = long_regex.sub(r'', string)
    return string

def cleaner(log_messages):
    clean_log = {}
    for key in log_messages:
        for item in log_messages[key]:
            #item =  " ".join(item.split('.'))
            item = re.sub(_line_number, "at line *", item)
            item = re.sub(_url, " ", item)
            item = re.sub(_filepath, " ", item)
            item = re.sub("\d+", " ", item)
            item = re.sub(r'\b\w\b', ' ', item)
            item = substitute_path(item)
            if key in clean_log.keys() and item not in clean_log[key]:
                clean_log[key] = clean_log[key] + ' ' + remove_whitespaces(item)
            else:
                clean_log[key] = remove_whitespaces(item)
    return clean_log

In [16]:
clustering_data = cleaner(clustering_data)

In [17]:
clustering_data

{0: 'SyntaxError',
 1: 'FileNotFoundError',
 2: 'RuntimeError',
 3: 'SyntaxError',
 4: 'cffi VerificationError CompileError',
 5: 'ERROR Failed building wheel for spacy',
 6: 'ERROR Failed building wheel for fastavro',
 7: 'ERROR Failed building wheel for pyeclib',
 8: 'NameError',
 9: 'SyntaxError',
 10: 'SyntaxError',
 11: 'ERROR Failed building wheel for thriftrw',
 12: 'ERROR No matching distribution found for',
 13: 'FileNotFoundError',
 14: 'SyntaxError',
 15: 'FileNotFoundError',
 16: 'ERROR No matching distribution found for itk segmentation',
 17: 'SyntaxError',
 18: 'ERROR No matching distribution found for mxnet mkl',
 19: 'ModuleNotFoundError',
 20: 'FileNotFoundError',
 21: 'ERROR No matching distribution found for wrapt',
 22: 'FileNotFoundError',
 23: 'FileNotFoundError',
 24: 'FileNotFoundError',
 25: 'ERROR No matching distribution found for mutagen',
 26: 'SyntaxError',
 27: 'AttributeError',
 28: 'SyntaxError',
 29: 'ERROR No matching distribution found for pwntools'

In [18]:
def tokenization(log_messages):
    tokenized = []
    for item in log_messages.values():
        if "for" in item:
            item = item.split("for",1)[0] 
        tokenized.append(TreebankWordTokenizer().tokenize(item))
    cleaned_tokens = []
    for id, row in enumerate(tokenized):
        #cleaned_tokens.append([i for i in row if i.lower() not in stop and not i.lower().isnumeric()])
        cleaned_tokens.append(list(filter(None, [i.strip()
                                                 for i in row 
                                                 if i not in solver_total_errors_df['package_name'][id]
                                                 and not i.lower().isnumeric()])))
    return cleaned_tokens

In [19]:
clean_clustering_data = tokenization(clustering_data)

In [20]:
len(clean_clustering_data)

1624

In [21]:
solver_total_errors_df['clustering_data'] = solver_total_errors_df['index'].map(clustering_data)

In [22]:
solver_total_errors_df['clean_clustering_data'] = clean_clustering_data

In [26]:
solver_total_errors_df.head(20)

,index,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout,Error_info,command_info,cwd,Complete_output,ERROR,specific_error,clustering_data,clean_clustering_data
0,0,tryton,3.2.20,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps tryton==3.2.20 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe n...",1.0,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe not found: installers can not be created, ""\\n ...",Collecting tryton==3.2.20\n Downloading https://files.pythonhosted.org/packages/8d/b6/a88e6e4df1557cf3f265b12fd585f36e212fd8669aa394c9fbb5a86e56d3/tryton-3.2.20.tar.gz (811kB)\n,60.0,Command exited with non zero status code 1 ERROR Command errored out with exit status 1,[command home solver venv bin python3 c import sys setuptools tokenize sys argv 0 tmp pip install gczw75i tryton setup py file tmp pip install gczw75i tryton setup py f getattr tokenize open open file code f read replace r n n f close exec compile code file exec egg info egg base tmp pip install gczw75i tryton pip egg info],[cwd tmp pip install gczw75i tryton],"[Complete output 6 lines, Traceback most recent call last, File string line 1 in module, File tmp pip install gczw75i tryton setup py line 289, print makensis exe not found installers can not be created, , SyntaxError Missing parentheses in call to print Did you mean print makensis exe not found installers can not be created]",[ERROR Command errored out with exit status 1 python setup py egg info Check the logs for full command output],[SyntaxError Missing parentheses in call to print Did you mean print makensis exe not found installers can not be created],SyntaxError,[SyntaxError]
1,1,pyobjc-framework-contacts,4.0b1,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps pyobjc-framework-contacts==4.0b1 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/setup.py'""'""'; __file__='""'""'/tmp/pip-install-t57p8sf3/pyobjc-framework-contacts/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __fil

In [27]:
with open("solver-errors-clean-clustering-data.txt", "wb") as fp: 
    pickle.dump(solver_total_errors_df['clean_clustering_data'], fp)

In [28]:
solver_total_errors_df.to_csv('solver-error-clean-data.csv', index=False)